In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
) 

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 650 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 31s - loss: 2.6302 - auc: 0.5407 - accuracy: 0.5286 - val_loss: 0.8065 - val_auc: 0.7991 - val_accuracy: 0.4875
Epoch 2/30
40/40 - 29s - loss: 0.9715 - auc: 0.5528 - accuracy: 0.5349 - val_loss: 1.0716 - val_auc: 0.8470 - val_accuracy: 0.5188
Epoch 3/30
40/40 - 30s - loss: 0.8516 - auc: 0.6028 - accuracy: 0.5636 - val_loss: 0.6265 - val_auc: 0.8708 - val_accuracy: 0.6047
Epoch 4/30
40/40 - 28s - loss: 0.7223 - auc: 0.6694 - accuracy: 0.6187 - val_loss: 0.7610 - val_auc: 0.8881 - val_accuracy: 0.4875
Epoch 5/30
40/40 - 28s - loss: 0.7515 - auc: 0.6684 - accuracy: 0.6211 - val_loss: 0.6414 - val_auc: 0.9013 - val_accuracy: 0.5156
Epoch 6/30
40/40 - 29s - loss: 0.7992 - auc: 0.6737 - accuracy: 0.6243 - val_loss: 0.4761 - val_auc: 0.9102 - val_accuracy: 0.7828
Epoch 7/30
40/40 - 28s - loss: 0.7092 - auc: 0.7071 - accuracy: 0.6424 - val_loss: 0.5533 - val_auc: 0.9208 - val_accuracy: 0.6750
Epoch 8/30
40/40 - 29s - loss: 0.6040 - auc: 0.7599 - accuracy: 0.6837 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

635
15


In [8]:
prediction

array([[0.9999267 ],
       [0.62109345],
       [0.9210288 ],
       [0.95330137],
       [0.7215743 ],
       [0.949877  ],
       [0.9731558 ],
       [0.93870443],
       [0.67968845],
       [0.6035732 ],
       [0.9997342 ],
       [0.9988543 ],
       [0.8549789 ],
       [0.99748176],
       [0.9814596 ],
       [0.9999964 ],
       [0.9998896 ],
       [0.73205477],
       [0.8549789 ],
       [0.8173748 ],
       [0.99987376],
       [0.9969085 ],
       [0.8928133 ],
       [0.48470515],
       [0.9935241 ],
       [0.6119376 ],
       [0.99818426],
       [0.57897395],
       [0.99113876],
       [0.792021  ],
       [0.67609763],
       [0.9829547 ],
       [0.803275  ],
       [0.8173748 ],
       [0.86618495],
       [0.6035732 ],
       [0.89159524],
       [0.7897041 ],
       [0.7763148 ],
       [0.8172894 ],
       [0.9919333 ],
       [0.8172894 ],
       [0.9950931 ],
       [0.7342413 ],
       [0.96137077],
       [0.8475347 ],
       [0.6218829 ],
       [0.923

In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

11/11 [==============================] - 5s 482ms/step - loss: 0.8070 - auc: 0.9752 - accuracy: 0.5154


[0.8070117235183716, 0.975236713886261, 0.5153846144676208]